<a href="https://colab.research.google.com/github/esmeee51/B134030044/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install -q pyngrok flask_ngrok # for colab
!pip install -q flask line-bot-sdk

In [ ]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('Ngrok_authtoken'))
    ngrok.connect(5000)
    run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage, StickerMessage, ImageMessage, VideoMessage, LocationMessage
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

import google.generativeai as genai

genai.configure(api_key=userdata.get("GENAI_KEY"))
model = genai.GenerativeModel("gemini-2.0-flash")
def ask_gemini(question):
  response = model.generate_content(question)
  return response.text

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
      line_bot_api = MessagingApi(api_client)

      action = event.message.text
      if action == 'sticker':
        #回貼圖
        reply = StickerMessage(
          package_id=6136,
          sticker_id=10551395
          )
      elif action == 'image':
        reply = ImageMessage(
            original_content_url='https://www.billboard.com/wp-content/uploads/2022/01/yuuri-billboard-japan-2022-billboard-1548.jpg?w=1024',
            preview_url='https://www.billboard.com/wp-content/uploads/2022/01/yuuri-billboard-japan-2022-billboard-1548.jpg?w=1024'
        )
      elif action == 'video':
        #回影片
        reply = VideoMessage(
            original_content_url='https://youtu.be/gRX3Gm-YPRY?feature=shared',
            preview_url='https://i.ytimg.com/vi/gRX3Gm-YPRY/hq720.jpg?sqp=-oaymwEnCNAFEJQDSFryq4qpAxkIARUAAIhCGAHYAQHiAQoIGBACGAY4AUAB&rs=AOn4CLCjDdnC-PboZLUiBdEVeEjuBtlEXA'
        )
      elif action == 'location':
        #回地點
        reply = LocationMessage(
            title='Taipei 101',
            address='110台北市信義區信義路五段7號',
            latitude='25.033611',
            longitude='121.564444'
        )
      else:
        #回問gemini
        reply = TextMessage(text=ask_gemini(action))

      line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    reply
                ]
            )
        )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://ef46-104-199-162-138.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [14/May/2025 13:01:24] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/May/2025 13:01:24] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/May/2025 13:01:37] "POST / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [14/May/2025 13:01:51] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/May/2025 13:02:20] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/May/2025 13:02:29] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/May/2025 13:02:47] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/May/2025 13:03:03] "POST /callback HTTP/1.1" 200 -
ERROR:__main__:Exception on /callback [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request